In [85]:
import pandas as pd
from openai import OpenAI
from pinecone import Pinecone
from keys.keys import OPENAI_API_KEY, PINECONE_API_KEY, INDEX_NAME
from functions.query_index import query_index
from functions.read_docs import read_docs
from functions.generate_answer import generate_answer
from functions.query import query
# from functions.query import query

In [86]:
client = OpenAI(api_key=OPENAI_API_KEY)
model = 'text-embedding-ada-002'

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

In [87]:
file_name = "/Users/tashatanarugsachock/Desktop/shopperbot-llm/dataset/exam.xlsx" 
df = pd.DataFrame(data=pd.read_excel(io=file_name))

df.head(5)

,Type,Question,A,B,C,D,Answer Key
0,General Information,When is Siam Paragon opening hours?,Siam Paragon opens from 8:00 AM to 8:00 PM.,Siam Paragon opens from 9:00 AM to 9:00 PM.,Siam Paragon opens from 10:00 AM to 10:00 PM.,Siam Paragon opens from 11:00 AM to 11:00 PM.,C
1,General Information,Is there free Wi-Fi available in Siam Paragon?,"No, there is no Wi-Fi available in Siam Paragon.","Yes, but it's only available for staff members.","Yes, there is free Wi-Fi available in Siam Par...","Yes, but customers need to pay for Wi-Fi acces...",C
2,General Information,Are pets allowed in Siam Paragon?,"Yes, pets are welcome in Siam Paragon.","No, pets are not allowed in Siam Paragon.","Yes, but only small pets are allowed in Siam P...","Yes, but only if they are carried in pet carri...",B
3,General Information,What is the contact number for Siam Paragon's ...,02-610-8000,02-937-1000,02-937-1345,02-215-1000,A
4,General Information,Are there any prayer rooms available?,"No, there are no prayer rooms available in Sia...","Yes, there is a Muslim prayer room available o...","Yes, there is a prayer room available on the 3...","Yes, there is a prayer room available on the 2...",B


In [88]:
base = 'From this question, what is the correct choice - answer in this format X: the answer? Question: '

In [89]:
def dfquery(row, client, model, index):
    q = row['Question']
    a = row['A']
    b = row['B']
    c = row['C']
    d = row['D']
    prompt = base + q + ' A: ' + a + ' B: ' + b + ' C: ' + c + ' D: ' + d
    
    var_fn = {}
    
    index_result = query_index(client=client, model=model, index=index, prompt=prompt)
    var_fn.update({'index_result': index_result})
    
    file_ids = []
    for res in index_result:
        file_id = res.split(': ')[1]
        file_ids.append(file_id)
        
    contents = read_docs(file_ids=file_ids)
    
    response = generate_answer(client=client, prompt=prompt, contents=contents)
    var_fn.update({'response': response})
    
    answer = var_fn.get('response', '')
    answer = answer.replace('\r', '').replace('\n', '')
    answer = answer.replace('Answer: ', '')
   
    return answer

In [90]:
df['GPT'] = df.apply(dfquery, args=(client, model, index), axis = 1)

In [121]:
def ansformat(row):
    ans = row['GPT']
    parts = ans.split(': ')
    
    if len(parts) > 2 and (ans[:6].count(":") == 2 or parts[0] == "The correct choice is"):
        return parts[1].replace('.','')
    else:
        return parts[0][-1]
    
    
# A: B: answer
# The correct choice is: A: 
# .A:
# A: time is 10:00
# A: B: time is 10:00
    

In [122]:
df['ansformatted'] = df.apply(ansformat, axis = 1)

In [124]:
def anscheck(row):
    
    if row['Answer Key'] == row['ansformatted']:
        return 1
    else:
        return 0

In [125]:
df['score'] = df.apply(anscheck, axis = 1)

In [127]:
df.groupby(['Type'])['score'].sum()


Type
ATM and Banking Services           3
Events and Promotion               3
General Information                5
Membership and Loyalty Program     1
Other Information                  9
Parking Services                   6
Public Transportation Access       4
Restaurant                        21
Restroom                           1
Store Information                 11
Name: score, dtype: int64

In [149]:
result = df.groupby('Type').agg({'score': 'sum', 'Type': 'count'})
result['percentage'] = (result['score'] / result['Type'] * 100).round(2)
result.columns = ['Correct Answers', 'Total Questions', 'Percentage']

In [151]:
result

,Correct Answers,Total Questions,Percentage
Type,,,
ATM and Banking Services,3,8,37.50
Events and Promotion,3,6,50.00
General Information,5,9,55.56
Membership and Loyalty Program,1,8,12.50
Other Information,9,11,81.82
Parking Services,6,13,46.15
Public Transportation Access,4,5,80.00
Restaurant,21,27,77.78
Restroom,1,6,16.67


In [152]:
result_path = "/Users/tashatanarugsachock/Desktop/shopperbot-llm/dataset/result/result01.xlsx"

In [153]:
df.to_excel(result_path, index=False)